<a href="https://colab.research.google.com/github/shamvrueth/genai-playground/blob/main/Code_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 849.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import requests
from IPython.display import Markdown, display, update_display
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gradio as gr

In [ ]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
QWEN = "Qwen/CodeQwen1.5-7B-Chat"

In [ ]:
SUPPORTED_LANGUAGES = sorted(["python", "javascript", "java", "c", "cpp", "go", "rust"])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(QWEN) #automatically detect and load correct tokenizer for the model
model = AutoModelForCausalLM.from_pretrained(QWEN, device_map="auto", torch_dtype="auto") #initializes correct architecture for the model

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.71G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread

def code_converter(code, source_lang, target_lang):
  prompt = (
    f"You are an expert code converter. "
    f"Translate the following {source_lang} code into {target_lang}. "
    f"Provide ONLY the translated code in a single, clean code block. Do not include explanations, comments, or any extra text."
    f"\n\n```{source_lang}\n{code}\n```"
  )
  messages = [
    {"role": "system", "content": "You are a helpful assistant that translates code accurately."},
    {"role": "user", "content": prompt}
  ]
  inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda") #convert chat messages to tokens
  streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True) #initializes streamer for streaming response of model
  generation_kwargs = {"input_ids": inputs, "streamer": streamer, "max_new_tokens": 2000, "do_sample": True, "top_p": 0.9, "temperature": 0.7}
  thread = Thread(target=model.generate, kwargs=generation_kwargs) #generation_kwargs = parameters for model.generate
  thread.start()
  response = ""
  for new_text in streamer:
    response += new_text
    yield response

In [ ]:
with gr.Blocks(theme=gr.themes.Soft(), title="AI Code Converter") as demo:
    with gr.Row():
        with gr.Column(scale=1):
            source_lang_dd = gr.Dropdown(
                label="Source Language",
                choices=SUPPORTED_LANGUAGES,
                value="python"
            )
            target_lang_dd = gr.Dropdown(
                label="Target Language",
                choices=SUPPORTED_LANGUAGES,
                value="javascript"
            )
            convert_btn = gr.Button("Convert Code", variant="primary", scale=1)

        with gr.Column(scale=2):
            source_code_input = gr.Code(
                label="Source Code",
                language="python",
                lines=20
            )
            converted_code_output = gr.Code(
                label="Converted Code",
                language="javascript",
                lines=20,
                interactive=False # Make it read-only
            )

    source_lang_dd.change(lambda lang: gr.update(language=lang), inputs=source_lang_dd, outputs=source_code_input)
    target_lang_dd.change(lambda lang: gr.update(language=lang), inputs=target_lang_dd, outputs=converted_code_output)

    # The main conversion action
    convert_btn.click(
        fn=code_converter,
        inputs=[source_code_input, source_lang_dd, target_lang_dd],
        outputs=converted_code_output,
        api_name="convert"
    )

    # Add examples for users to try
    gr.Examples(
        examples=[
            [
                "def factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)",
                "python",
                "go"
            ],
            [
                "public class HelloWorld {\n    public static void main(String[] args) {\n        System.out.println(\"Hello, World!\");\n    }\n}",
                "java",
                "rust"
            ],
        ],
        inputs=[source_code_input, source_lang_dd, target_lang_dd],
        outputs=converted_code_output,
        fn=code_converter,
        cache_examples=True
    )

In [ ]:
demo.launch(share=False)

Caching examples at: '/content/.gradio/cached_examples/22'
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>